# Low Rank Training and evaluation script

1) Install requirements (Linux and Mac)

Open console and type

```bash
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
```

Then select the appropriate jupyter kernel

Or install directly in jupyter


In [1]:
!pip install --quiet torch torchvision matplotlib pandas tqdm wandb tensorly tensorly-torch tntorch transformers


2) Download the data (Cifar10 test case)

In [2]:
from dataset.data_adversarial_rs.create_cifar10 import save_cifar10_images

save_cifar10_images("dataset/data_adversarial_rs/Cifar10")

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 170M/170M [01:32<00:00, 1.84MB/s]
Processing ../../Cifar10_test.txt: 100%|████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 2716.86it/s]


2) Train the network

You can directly attack with pre-trained networks using run_train=false below

In [5]:
%%bash
#wandb login --relogin

run_train=true #
run_attack=true

iterations="1"

#regularization
regularizer_betas="0.0 0.15" # 0.05 0.1 0.15 0.2"

# adversarial attack
attack_f_1="condlr_fgsm"
epsilons_1="0.002 0.004 0.006"

# Model settings
models="vgg16" 
target_model="vgg16"
dataset=3
crop_size=32


#DLRT parameters
tolerances="0.05" # Good default value
rmax=150 # Good default value
num_local_iter=10  # Good default value
r_init=50 # Good default value

#training parameters
train_batch_size=128
val_batch_size=128
weight_decay=0.0 
num_epochs=30
num_epochs_ft=2
#logging
wandb=0  # 1 for enabled, 0 for disabled
wandb_tag="google_colab_example"

# Loop over all combinations of models and pretrained weights
for i in $(seq 1 $iterations); do
echo "iteration $i"
for model in $models; do
for tol in $tolerances; do
for beta in $regularizer_betas; do
    if [ "$run_train" = true ]; then
        python pretrain_cls_low_rank_robustness.py \
            --dataID $dataset \
            --num_local_iter $num_local_iter \
            --rmax $rmax \
            --init_r $r_init \
            --lr 1e-4 \
            --num_epochs $num_epochs \
            --num_epochs_low_rank_ft $num_epochs_ft \
            --num_local_iter $num_local_iter \
            --network $model \
            --save_name "$beta" \
            --tol "$tol" \
            --wandb $wandb \
            --wandb_tag "$wandb_tag" \
            --robusteness_regularization_beta "$beta" \
            --val_batch_size $val_batch_size  \
            --train_batch_size $train_batch_size \
            --crop_size $crop_size \
            --weight_decay $weight_decay \
            --print_per_batches 100 \
            --load_model 0 \
            --root_dir ./ \
            --save_path_prefix ./model_lr/
    fi
    if [ "$run_attack" = true ]; then
        for eps_attack in $epsilons_1; do
            echo "Delete all attacked images for a fresh start"
            if [ "$dataset" = "3" ]; then
                echo "Deleting ./dataset/data_adversarial_rs/Cifar10_adv/$attack_f_1/low_rank/$model/*.png"
                rm ./dataset/data_adversarial_rs/Cifar10_adv/$attack_f_1/low_rank/$model/*.png
            fi
            if [ "$dataset" = "2" ]; then
                echo "Deleting ./dataset/data_adversarial_rs/AID_adv/$attack_f_1/low_rank/$model/*.png"
                rm ./dataset/data_adversarial_rs/AID_adv/$attack_f_1/low_rank/$model/*.png
            fi
            if [ "$dataset" = "1" ]; then
                echo "Deleting ./dataset/data_adversarial_rs/UCM_adv/$attack_f_1/low_rank/$model/*.png"
                rm ./dataset/data_adversarial_rs/UCM_adv/$attack_f_1/low_rank/$model/*.png
            fi
          
            python attack_cls_low_rank.py \
                --dataID $dataset \
                --rmax $rmax \
                --init_r $r_init \
                --tol "$tol" \
                --robusteness_regularization_beta "$beta" \
                --attack_func $attack_f_1 \
                --network $model \
                --crop_size $crop_size \
                --epsilon $eps_attack \
                --save_path_prefix ./  \
                --model_root_dir ./model_lr/
                         
            python test_cls_low_rank.py \
                --dataID $dataset \
                --rmax $rmax \
                --init_r $r_init \
                --tol "$tol" \
                --robusteness_regularization_beta "$beta" \
                --target_network $target_model \
                --surrogate_network $model \
                --attack_func $attack_f_1 \
                --wandb $wandb \
                --wandb_tag "$wandb_tag" \
                --attack_epsilon $eps_attack \
                --crop_size $crop_size \
                --val_batch_size $val_batch_size \
                --root_dir ./ \
                --save_path_prefix ./model_lr/
        done
    fi
done
done
done
done


wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: ERROR Find detailed error logs at: /home/8v5/Desktop/adversarial_rs_low_rank/last_test/supplementary_material/wandb/debug-cli.8v5.log
Error: api_key not configured (no-tty). call wandb login [your_api_key]


iteration 1
Network: vgg16
DataParallel(
  (module): VGG(
    (features): Sequential(
      (0): CustomConv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ReLU(inplace=True)
      (2): Conv2dLowRankLayerAugBUG(
        rank=[32, 32, 3, 3]
        (Us): ParameterList(
            (0): Parameter containing: [torch.float32 of size 64x32 (cuda:0)]
            (1): Parameter containing: [torch.float32 of size 64x32 (cuda:0)]
            (2): Parameter containing: [torch.float32 of size 3x3 (cuda:0)]
            (3): Parameter containing: [torch.float32 of size 3x3 (cuda:0)]
        )
      )
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2dLowRankLayerAugBUG(
        rank=[64, 32, 3, 3]
        (Us): ParameterList(
            (0): Parameter containing: [torch.float32 of size 128x64 (cuda:0)]
            (1): Parameter containing: [torch.float32 of size 64x32 (cuda:0)]
   

/home/8v5/Desktop/adversarial_rs_low_rank/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Step 99; Epoch 1/30:  100/391 Time: 0.08 cls_loss = 1.618 acc = 0.405 cr = 94.14

Step 199; Epoch 1/30:  200/391 Time: 0.08 cls_loss = 1.023 acc = 0.645 cr = 94.34

Step 299; Epoch 1/30:  300/391 Time: 0.08 cls_loss = 0.870 acc = 0.700 cr = 94.43

Step 399; Epoch 2/30:  9/391 Time: 0.08 cls_loss = 0.778 acc = 0.732 cr = 93.75

Step 499; Epoch 2/30:  109/391 Time: 0.08 cls_loss = 0.721 acc = 0.750 cr = 93.75

Step 599; Epoch 2/30:  209/391 Time: 0.08 cls_loss = 0.708 acc = 0.756 cr = 93.75

Step 699; Epoch 2/30:  309/391 Time: 0.07 cls_loss = 0.675 acc = 0.769 cr = 93.75

Step 799; Epoch 3/30:  18/391 Time: 0.07 cls_loss = 0.661 acc = 0.773 cr = 93.75

Step 899; Epoch 3/30:  118/391 Time: 0.08 cls_loss = 0.613 acc = 0.796 cr = 93.75

Step 999; Epoch 3/30:  218/391 Time: 0.08 cls_loss = 0.608 acc = 0.794 cr = 93.75

Step 1099; Epoch 3/30:  318/391 Time: 0.08 cls_loss = 0.578 acc = 0.804 cr = 93.75

Step 1199; Epoch 4/30:  27/391 Time: 0.08 cls_loss = 0.553 acc = 0.810 cr = 93.75

Step 12

rm: cannot remove './dataset/data_adversarial_rs/Cifar10_adv/condlr_fgsm/low_rank/vgg16/*.png': No such file or directory


...

Loading data...
./Cifar10_adv/condlr_fgsm/low_rank/vgg16/
./dataset/Cifar10_test.txt
----
[64, 3, 3, 3]
10
[32, 30, 3, 3]
[62, 32, 3, 3]
[64, 64, 3, 3]
[115, 64, 3, 3]
[123, 109, 3, 3]
[125, 115, 3, 3]
[146, 106, 3, 3]
[143, 128, 3, 3]
[142, 132, 3, 3]
[147, 130, 3, 3]
[140, 131, 3, 3]
[133, 133, 3, 3]
146
137
Epoch[1]-Validation-[10/79] Batch OA: 89.84 %
Epoch[1]-Validation-[20/79] Batch OA: 85.16 %
Epoch[1]-Validation-[30/79] Batch OA: 86.72 %
Epoch[1]-Validation-[40/79] Batch OA: 89.06 %
Epoch[1]-Validation-[50/79] Batch OA: 95.31 %
Epoch[1]-Validation-[60/79] Batch OA: 87.50 %
Epoch[1]-Validation-[70/79] Batch OA: 90.62 %
---------------Accuracy of     airplane : 92.40 %---------------
---------------Accuracy of   automobile : 95.70 %---------------
---------------Accuracy of         bird : 86.50 %---------------
---------------Accuracy of          cat : 74.70 %---------------
---------------Accuracy of         deer : 89.00 %---------------
---------------Accuracy of          dog :

rm: cannot remove './dataset/data_adversarial_rs/Cifar10_adv/condlr_fgsm/low_rank/vgg16/*.png': No such file or directory


Network: vgg16
load model from
./model_lr/Cifar10/Pretrain/low_rank/vgg16/beta_0.15_tol_0.05_rmax_150.0_init_rank_50.0.pth
Low rank layer ranks
[64, 3, 3, 3]
10
[32, 30, 3, 3]
[62, 32, 3, 3]
[64, 64, 3, 3]
[115, 64, 3, 3]
[123, 109, 3, 3]
[125, 115, 3, 3]
[146, 106, 3, 3]
[143, 128, 3, 3]
[142, 132, 3, 3]
[147, 130, 3, 3]
[140, 131, 3, 3]
[133, 133, 3, 3]
146
137
------


Batch: 10000/10000: 100%|██████████| 10000/10000 [03:22<00:00, 49.45it/s]


Loading data...
./Cifar10_adv/condlr_fgsm/low_rank/vgg16/
./dataset/Cifar10_test.txt
----
[64, 3, 3, 3]
10
[32, 30, 3, 3]
[62, 32, 3, 3]
[64, 64, 3, 3]
[115, 64, 3, 3]
[123, 109, 3, 3]
[125, 115, 3, 3]
[146, 106, 3, 3]
[143, 128, 3, 3]
[142, 132, 3, 3]
[147, 130, 3, 3]
[140, 131, 3, 3]
[133, 133, 3, 3]
146
137
Epoch[1]-Validation-[10/79] Batch OA: 89.84 %
Epoch[1]-Validation-[20/79] Batch OA: 85.16 %
Epoch[1]-Validation-[30/79] Batch OA: 86.72 %
Epoch[1]-Validation-[40/79] Batch OA: 89.06 %
Epoch[1]-Validation-[50/79] Batch OA: 95.31 %
Epoch[1]-Validation-[60/79] Batch OA: 87.50 %
Epoch[1]-Validation-[70/79] Batch OA: 90.62 %
---------------Accuracy of     airplane : 92.40 %---------------
---------------Accuracy of   automobile : 95.70 %---------------
---------------Accuracy of         bird : 86.50 %---------------
---------------Accuracy of          cat : 74.70 %---------------
---------------Accuracy of         deer : 89.00 %---------------
---------------Accuracy of          dog :

3) Run the full rank baseline models as comparison. 

In [6]:
%%bash
run_train=true
run_attack=true

iterations="1" # if you want to compute the mean over multiple runs, increase iteration count


regularizer_betas="0.0"

attack_f_1="condlr_fgsm" # l1 FGSM attack
epsilons_1="0.002 0.004 0.006"



models="vgg16" #"alexnet vgg11 vgg16 vgg19 inception resnet18 resnet50 resnet101 resnext50_32x4d resnext101_32x8d densenet121 densenet169 densenet201 regnet_x_400mf regnet_x_8gf regnet_x_16gf"
target_model="vgg16"

dataset=3
crop_size=32

#training parameters
train_batch_size=128
val_batch_size=128
num_epochs=30
weight_decay=0.0 

# Enable or disable WandB logging
wandb=0 # 1 for enabled, 0 for disabled
wandb_tag="google_colab_example"

# Loop over all combinations of models and pretrained weights
for i in $(seq 1 $iterations); do
echo "iteration $i"
for model in $models; do
for beta in $regularizer_betas; do
    if [ "$run_train" = true ]; then
        python pretrain_cls_robustness.py \
            --dataID $dataset \
            --lr 1e-4 \
            --num_epochs $num_epochs \
            --network $model \
            --save_name "$beta" \
            --wandb $wandb \
            --wandb_tag "$wandb_tag" \
            --robusteness_regularization_beta "$beta" \
            --train_batch_size $train_batch_size \
            --val_batch_size $val_batch_size \
            --crop_size $crop_size \
            --weight_decay $weight_decay \
            --print_per_batches 100 \
            --root_dir ./
    fi
    if [ "$run_attack" = true ]; then
        # first attack
        for eps_attack in $epsilons_1; do
            echo "Delete all attacked images for a fresh start"
            if [ "$dataset" = "3" ]; then
                echo "Deleting ./dataset/data_adversarial_rs/Cifar10_adv/$attack_f_1/baseline/$model/*.png"
                rm ./dataset/data_adversarial_rs/Cifar10_adv/$attack_f_1/baseline/$model/*.png
            fi
            if [ "$dataset" = "2" ]; then
                echo "Deleting ./dataset/data_adversarial_rs/AID_adv/$attack_f_1/baseline/$model/*.png"
                rm ./dataset/data_adversarial_rs/AID_adv/$attack_f_1/baseline/$model/*.png
            fi
            if [ "$dataset" = "1" ]; then
                echo "Deleting ./dataset/data_adversarial_rs/UCM_adv/$attack_f_1/baseline/$model/*.png"
                rm ./dataset/data_adversarial_rs/UCM_adv/$attack_f_1/baseline/$model/*.png
            fi
          
            python attack_cls.py \
                --robusteness_regularization_beta "$beta" \
                --dataID $dataset \
                --attack_func $attack_f_1 \
                --network $model \
                --epsilon $eps_attack \
                --crop_size $crop_size \
                --save_path_prefix ./
            
            python test_cls.py \
                --robusteness_regularization_beta "$beta" \
                --dataID $dataset \
                --target_network $target_model \
                --surrogate_network $model \
                --attack_func $attack_f_1 \
                --wandb $wandb \
                --wandb_tag "$wandb_tag" \
                --attack_epsilon $eps_attack \
                --crop_size $crop_size \
                --val_batch_size $val_batch_size \
                --root_dir ./
        done
    fi
done
done
done 

iteration 1
[INFO] Saving path for the pre-trained model:  ./models/Cifar10/Pretrain/baseline/vgg16/
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1

/home/8v5/Desktop/adversarial_rs_low_rank/venv/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/8v5/Desktop/adversarial_rs_low_rank/venv/lib/python3.10/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Step 99; Epoch 1/30:  99/391 Time: nan cls_loss = nan acc = nan

Step 199; Epoch 1/30:  199/391 Time: 0.02 cls_loss = 0.734 acc = 0.757

Step 299; Epoch 1/30:  299/391 Time: 0.02 cls_loss = 0.635 acc = 0.788

Step 399; Epoch 2/30:  8/391 Time: 0.02 cls_loss = 0.564 acc = 0.812

Step 499; Epoch 2/30:  108/391 Time: 0.02 cls_loss = 0.493 acc = 0.834

Step 599; Epoch 2/30:  208/391 Time: 0.02 cls_loss = 0.454 acc = 0.851

Step 699; Epoch 2/30:  308/391 Time: 0.02 cls_loss = 0.452 acc = 0.846

Step 799; Epoch 3/30:  17/391 Time: 0.02 cls_loss = 0.435 acc = 0.852

Step 899; Epoch 3/30:  117/391 Time: 0.02 cls_loss = 0.363 acc = 0.878

Step 999; Epoch 3/30:  217/391 Time: 0.02 cls_loss = 0.360 acc = 0.882

Step 1099; Epoch 3/30:  317/391 Time: 0.02 cls_loss = 0.361 acc = 0.879

Step 1199; Epoch 4/30:  26/391 Time: 0.02 cls_loss = 0.316 acc = 0.893

Step 1299; Epoch 4/30:  126/391 Time: 0.02 cls_loss = 0.284 acc = 0.902

Step 1399; Epoch 4/30:  226/391 Time: 0.02 cls_loss = 0.290 acc = 0.902


rm: cannot remove './dataset/data_adversarial_rs/Cifar10_adv/condlr_fgsm/baseline/vgg16/*.png': No such file or directory


Network: vgg16
load model from
./models/Cifar10/Pretrain/baseline/vgg16/beta_0.0.pth


...

Loading data...
./Cifar10_adv/condlr_fgsm/baseline/vgg16/
./dataset/Cifar10_test.txt
----
Epoch[1]-Validation-[10/79] Batch OA: 92.97 %
Epoch[1]-Validation-[20/79] Batch OA: 92.19 %
Epoch[1]-Validation-[30/79] Batch OA: 93.75 %
Epoch[1]-Validation-[40/79] Batch OA: 90.62 %
Epoch[1]-Validation-[50/79] Batch OA: 96.09 %
Epoch[1]-Validation-[60/79] Batch OA: 89.06 %
Epoch[1]-Validation-[70/79] Batch OA: 93.75 %
---------------Accuracy of     airplane : 93.30 %---------------
---------------Accuracy of   automobile : 96.30 %---------------
---------------Accuracy of         bird : 87.90 %---------------
---------------Accuracy of          cat : 81.70 %---------------
---------------Accuracy of         deer : 90.50 %---------------
---------------Accuracy of          dog : 83.40 %---------------
---------------Accuracy of         frog : 94.50 %---------------
---------------Accuracy of        horse : 92.90 %---------------
---------------Accuracy of         ship : 96.00 %---------------
---

rm: cannot remove './dataset/data_adversarial_rs/Cifar10_adv/condlr_fgsm/baseline/vgg16/*.png': No such file or directory


Network: vgg16
load model from
./models/Cifar10/Pretrain/baseline/vgg16/beta_0.0.pth


Batch: 10000/10000: 100%|██████████| 10000/10000 [00:59<00:00, 166.98it/s]


Loading data...
./Cifar10_adv/condlr_fgsm/baseline/vgg16/
./dataset/Cifar10_test.txt
----
Epoch[1]-Validation-[10/79] Batch OA: 92.97 %
Epoch[1]-Validation-[20/79] Batch OA: 92.19 %
Epoch[1]-Validation-[30/79] Batch OA: 93.75 %
Epoch[1]-Validation-[40/79] Batch OA: 90.62 %
Epoch[1]-Validation-[50/79] Batch OA: 96.09 %
Epoch[1]-Validation-[60/79] Batch OA: 89.06 %
Epoch[1]-Validation-[70/79] Batch OA: 93.75 %
---------------Accuracy of     airplane : 93.30 %---------------
---------------Accuracy of   automobile : 96.30 %---------------
---------------Accuracy of         bird : 87.90 %---------------
---------------Accuracy of          cat : 81.70 %---------------
---------------Accuracy of         deer : 90.50 %---------------
---------------Accuracy of          dog : 83.40 %---------------
---------------Accuracy of         frog : 94.50 %---------------
---------------Accuracy of        horse : 92.90 %---------------
---------------Accuracy of         ship : 96.00 %---------------
---